In [47]:
# DO NOT USE SOS 2020 VOTER HISTORY FILE FROM WEBSITE. IT IS MISSING VOTERS.
# USE "voter_history_2020_nov3_elex_only_pullednov30" from data warehouse

In [48]:
import pandas as pd
import datetime
import mariadb
import numpy as np
import os
from pathlib import Path

In [49]:
# make sure these values are correct for each iteration of this notebook
# all other fields should run without changes
election_date = '2020-11-03'
election_type1 = 'GENERAL ELECTION'
election_type2 = 'None'
election_type3 = 'None'
sql_query = "SELECT * FROM `ga_sos_voters`.`voter_history_2020_nov3_elex_only_pullednov30`"
output_name = "general_2020.csv"


In [50]:
# Connect to data warehouse with MariaDB
try:
    conn = mariadb.connect(
        user=os.getenv("MARIADB_USER"),
        password=os.getenv("MARIADB_PASSWORD"),
        host=os.getenv("MARIADB_HOST"),
        port=int(os.getenv("MARIADB_PORT"))
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [51]:
# pull contents of SQL table into Pandas DataFrame
original_table = pd.read_sql(sql_query, conn, dtype='str')

In [ ]:
# make copy to avoid having to redownload as often
df = original_table.copy()

In [ ]:
df.head(20)

,ajc_id_num,county_num,registration_num,election_date,election_type,party,absentee,provisional,supplemental,data_from_history_year_file,ajc_data_acquisition_year,ajc_data_loader_initials
0,1,019,00591608,2020-11-03,003,,Y,N,N,2020,2020,JLP
1,2,027,00430264,2020-11-03,003,,Y,N,N,2020,2020,JLP
2,3,001,00652737,2020-11-03,003,,Y,N,N,2020,2020,JLP
3,4,023,05511334,2020-11-03,003,,Y,N,N,2020,2020,JLP
4,5,011,04059627,2020-11-03,003,,Y,N,N,2020,2020,JLP
5,6,028,03157858,2020-11-03,003,,Y,N,N,2020,2020,JLP
6,7,028,07321730,2020-11-03,003,,Y,N,N,2020,2020,JLP
7,8,007,06991394,2020-11-03,003,,Y,N,N,2020,2020,JLP
8,9,001,00653967,2020-11-03,003,,Y,N,N,2020,2020,JLP
9,10,023,11326919,2020-11-03,003,,Y,N,N,2020,2020,JLP


In [ ]:
# pull county code list and make dictionary
county_code_df = pd.read_sql("SELECT * FROM `ga_sos_voters`.`lu_countycode`", conn, dtype='str')

/var/folders/2d/dn3_f8x10wzf75hxkv2506g00000gp/T/ipykernel_73239/470233134.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  county_code_df = pd.read_sql("SELECT * FROM `ga_sos_voters`.`lu_countycode`", conn, dtype='str')


In [ ]:
county_code_dict = dict(zip(county_code_df.COUNTY_CODE, county_code_df.COUNTY))

In [ ]:
# make function to use dictionaries to fill in DataFrame columns with expanded information
def dict_lookup_list(column, dictionary):
    county_list = []
    for entry in column:
        if len(entry) == 3:
            county_name = dictionary.get(f'{entry}')
            county_list.append(county_name)
        else:
            county_list.append(entry)
    return county_list

In [ ]:
# use dict_lookup_list() to fill in 'county_name' and get rid of 'county_num'
if 'county_num' in df.columns:
    df.insert(0, 'county_name', dict_lookup_list(df.county_num, county_code_dict))

In [ ]:
df.pop('county_num')

0          019
1          027
2          001
3          023
4          011
          ... 
5001379    149
5001380    154
5001381    156
5001382    155
5001383    149
Name: county_num, Length: 5001384, dtype: object

In [ ]:
# rename voter_registration_number column to fit other data warehouse tables
df = df.rename(columns={'registration_num': 'voter_registration_number'})

In [ ]:
# function to add index to new DataFrame
def add_row_names(column):
    counter = 0
    row_name_list = []
    for row in column:
        counter += 1
        row_name_list.append(counter)
    return row_name_list

In [ ]:
if 'row_names' not in df.columns:
    df.insert(0, 'row_names', add_row_names(df['county_name']))

In [ ]:
# make dictionary of election types
election_type_df = pd.read_sql("SELECT * FROM `ga_sos_voters`.`lu_election_type`", conn)

/var/folders/2d/dn3_f8x10wzf75hxkv2506g00000gp/T/ipykernel_73239/2221766173.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  election_type_df = pd.read_sql("SELECT * FROM `ga_sos_voters`.`lu_election_type`", conn)


In [ ]:
election_type_dict = dict(zip(election_type_df.type_of_election, election_type_df.description))

In [ ]:
# Make election types all caps to match other tables
df['election_type'] = dict_lookup_list(df['election_type'], election_type_dict)

In [ ]:
df['election_type'] = df['election_type'].str.upper()

In [ ]:
# get rid of "None" values to match other tables in data warehouse

def remove_nones(column):
    entry_list = []
    for entry in column:
        if entry == "None":
            entry_list.append("")
        else:
            entry_list.append(entry)
    return entry_list

In [ ]:
df['county_name'] = remove_nones(df['county_name'])

In [ ]:
df['voter_registration_number'] = remove_nones(df['voter_registration_number'])

In [ ]:
df['election_date'] = remove_nones(df['election_date'])

In [ ]:
df['election_type'] = remove_nones(df['election_type'])

In [ ]:
df['party'] = remove_nones(df['party'])

In [ ]:
df['absentee'] = remove_nones(df['absentee'])

In [ ]:
df['provisional'] = remove_nones(df['provisional'])

In [ ]:
df['supplemental'] = remove_nones(df['supplemental'])

In [ ]:
# add or delete columns not used in other tables
if 'ballot_style' not in df.columns:
    df.insert(6, 'ballot_style', '')

In [ ]:
if 'data_from_history_year_file' in df.columns:
    df.pop('data_from_history_year_file')

In [ ]:
if 'ajc_data_acquisition_year' in df.columns:
    df.pop('ajc_data_acquisition_year')

In [ ]:
if 'ajc_data_loader_initials' in df.columns:
    df.pop('ajc_data_loader_initials')

In [ ]:
# there are both "GENERAL ELECTION" values and "GENERAL/SPECIAL ELECTION" values, and the vast majority are not duplicates
# I will use both values to filter and dedupe so that I am not missing 1/3 of the voters
# it would be a good idea to run this same notebook on more recent tables to check how this process compares to what the state does to get their values filtered
df.election_type.value_counts()

election_type
GENERAL ELECTION    5001384
Name: count, dtype: int64

In [ ]:
df[df['election_type'] == 'GENERAL ELECTION'].party.value_counts()

party
    5001384
Name: count, dtype: int64

In [ ]:
filtered_date_df = df[df['election_date'] == election_date]

In [ ]:
filtered_election_df = df[(df['election_type'] == election_type1) | (df['election_type'] == election_type2) | (df['election_type'] == election_type3)]

In [ ]:
filtered_date_election_df = filtered_election_df[filtered_election_df['election_date'] == election_date]

In [ ]:
filtered_date_election_df.row_names = add_row_names(filtered_date_election_df['county_name'])

In [ ]:
# it is hard to tell why there are duplicates with the general and general/special election types
filtered_date_election_df[filtered_date_election_df['voter_registration_number'].duplicated(keep=False)].sort_values('voter_registration_number')

,row_names,county_name,ajc_id_num,voter_registration_number,election_date,election_type,ballot_style,party,absentee,provisional,supplemental


In [ ]:
deduped_df = filtered_date_election_df.drop_duplicates('voter_registration_number')

In [ ]:
# there are 114 duplicated entries, and after deduping the DataFrame I have 114 fewer entries
len(filtered_date_election_df) - len(deduped_df)

0

In [ ]:
len(filtered_date_election_df[filtered_date_election_df['voter_registration_number'].duplicated()])

0

In [ ]:
len(filtered_date_election_df.drop_duplicates('voter_registration_number'))

5001384

In [ ]:
filtered_date_election_df.party.value_counts()

party
    5001384
Name: count, dtype: int64

In [ ]:
filtered_date_election_df.value_counts(['election_type'])

election_type   
GENERAL ELECTION    5001384
Name: count, dtype: int64

In [ ]:
output_path = Path().cwd().parent.joinpath(f'output_csv/{output_name}')
filtered_date_election_df.to_csv(output_path)